In [1]:
from matplotlib import pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd

In [2]:
ay_2019 = pd.read_csv("../Resources/2019.csv")
ay_2018 = pd.read_csv("../Resources/2018.csv")
ay_2017 = pd.read_csv("../Resources/2017.csv")
ay_2016 = pd.read_csv("../Resources/2016.csv")
ay_2015 = pd.read_csv("../Resources/2015.csv")
ay_2014 = pd.read_csv("../Resources/2014.csv")
ay_2013 = pd.read_csv("../Resources/2013.csv")
ay_2012 = pd.read_csv("../Resources/2012.csv")
ay_2011 = pd.read_csv("../Resources/2011.csv")
ay_2010 = pd.read_csv("../Resources/2010.csv")
ay_2009 = pd.read_csv("../Resources/2009.csv")

df_list = [ay_2019, ay_2018, ay_2017, ay_2016, ay_2015, ay_2014, 
            ay_2013, ay_2012, ay_2011, ay_2010, ay_2009]

#ay_2014.head()

df_list

[     Unnamed: 0        full_name position team  targets  rec  rec_yards  \
 0             1   Michael Thomas       WR   NO      185  149       1725   
 1             2     Chris Godwin       WR   TB      121   86       1333   
 2             3      Julio Jones       WR  ATL      157   99       1394   
 3             4      Cooper Kupp       WR   LA      134   94       1161   
 4             5  DeAndre Hopkins       WR  HOU      150  104       1165   
 ..          ...              ...      ...  ...      ...  ...        ...   
 198         199   Juwann Winfree       WR  DEN        0    0          0   
 199         200     Mack Hollins       WR  MIA        1    0          0   
 200         201   Matthew Slater       WR   NE        0    0          0   
 201         202    Taywan Taylor       WR  CLE        1    0          0   
 202         203     Trevor Davis       WR  MIA        0    0          0   
 
      air_yards  yac  td  adot  racr  ms_air  tgt_share  wopr    ppr  
 0         1488

In [3]:
###ADD YEAR###

year = 2019

for df in df_list:
    
    df["Year"] = year
    
    year -= 1

df_list[4]

,Unnamed: 0,full_name,position,team,targets,rec,rec_yards,air_yards,yac,td,adot,racr,ms_air,tgt_share,wopr,ppr,Year
0,1,Antonio Brown,WR,PIT,193,136,1834,2123,587,10,11.0,0.86,0.36,0.33,0.74,382.2,2015
1,2,Julio Jones,WR,ATL,203,136,1871,2063,641,8,10.2,0.91,0.43,0.33,0.79,371.1,2015
2,3,Brandon Marshall,WR,NYJ,173,109,1502,2096,442,14,12.1,0.72,0.38,0.29,0.69,343.2,2015
3,4,DeAndre Hopkins,WR,HOU,191,111,1521,2674,231,11,14.0,0.57,0.45,0.31,0.78,329.1,2015
4,5,Odell Beckham,WR,NYG,158,96,1449,1829,589,13,11.6,0.79,0.41,0.27,0.69,319.2,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,123,Matthew Slater,WR,NE,1,0,0,34,0,0,34.0,0.00,0.01,0.00,0.01,0.0,2015
123,124,Neal Sterling,WR,JAC,0,0,0,0,0,0,NaN,NaN,0.00,0.00,0.00,0.0,2015
124,125,Quan Bray,WR,IND,0,0,0,0,0,0,NaN,NaN,0.00,0.00,0.00,0.0,2015
125,126,Vince Mayle,WR,DAL,1,0,0,23,0,0,23.0,0.00,0.10,0.03,0.11,0.0,2015


In [4]:
merged_df = pd.concat(df_list)


In [5]:
merged_df

,Unnamed: 0,full_name,position,team,targets,rec,rec_yards,air_yards,yac,td,adot,racr,ms_air,tgt_share,wopr,ppr,Year
0,1,Michael Thomas,WR,NO,185,149,1725,1488,583,9,8.0,1.16,0.40,0.32,0.75,374.6,2019
1,2,Chris Godwin,WR,TB,121,86,1333,1301,577,9,10.8,1.02,0.23,0.22,0.49,274.1,2019
2,3,Julio Jones,WR,ATL,157,99,1394,1913,364,6,12.2,0.73,0.37,0.25,0.63,274.1,2019
3,4,Cooper Kupp,WR,LA,134,94,1161,1007,538,10,7.5,1.15,0.21,0.21,0.46,270.5,2019
4,5,DeAndre Hopkins,WR,HOU,150,104,1165,1580,387,7,10.5,0.74,0.34,0.30,0.69,264.3,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,12,Jordy Nelson,WR,GB,31,22,320,374,89,2,12.1,0.86,0.09,0.07,0.17,66.0,2009
12,13,Darrius Heyward-Bey,WR,OAK,40,9,124,637,12,1,15.9,0.19,0.22,0.13,0.35,29.3,2009
13,14,Eric Weems,WR,ATL,6,6,50,28,22,2,4.7,1.79,0.01,0.01,0.02,28.3,2009
14,15,Brandon Tate,WR,NE,2,0,0,73,0,0,36.5,0.00,0.12,0.03,0.13,1.1,2009


In [7]:
###Clean DF for ML###

cleaned_df = merged_df.drop(columns=["full_name", "Unnamed: 0", "position", "team"])

cleaned_df

,targets,rec,rec_yards,air_yards,yac,td,adot,racr,ms_air,tgt_share,wopr,ppr,Year
0,185,149,1725,1488,583,9,8.0,1.16,0.40,0.32,0.75,374.6,2019
1,121,86,1333,1301,577,9,10.8,1.02,0.23,0.22,0.49,274.1,2019
2,157,99,1394,1913,364,6,12.2,0.73,0.37,0.25,0.63,274.1,2019
3,134,94,1161,1007,538,10,7.5,1.15,0.21,0.21,0.46,270.5,2019
4,150,104,1165,1580,387,7,10.5,0.74,0.34,0.30,0.69,264.3,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,31,22,320,374,89,2,12.1,0.86,0.09,0.07,0.17,66.0,2009
12,40,9,124,637,12,1,15.9,0.19,0.22,0.13,0.35,29.3,2009
13,6,6,50,28,22,2,4.7,1.79,0.01,0.01,0.02,28.3,2009
14,2,0,0,73,0,0,36.5,0.00,0.12,0.03,0.13,1.1,2009
